<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/repair_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Navigate to project folder and ensure we're on the 'develop' branch
import os, subprocess, sys

REPO_DIR = "/content/Cloud_Computing"
if not os.path.isdir(REPO_DIR):
    subprocess.run([
        "git", "clone", "-b", "develop",
        "https://github.com/Method-for-Software-System-Development/Cloud_Computing.git",
        REPO_DIR
    ], check=True)

# Update & checkout develop if repo already exists
subprocess.run(["git", "-C", REPO_DIR, "fetch", "origin"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "checkout", "develop"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "pull"], check=True)

# Install importnb (only needed once per session)
%pip install importnb --quiet

# Add firebase folder to Python path and import its notebook as module
sys.path.append(os.path.join(REPO_DIR, "firebase"))
from importnb import Notebook

with Notebook():
    import FireBase as fb

from datetime import datetime



In [ ]:
def start_repair(sensor: str):
    """
    Logs the start time of a repair process.
    This function does not give points – only starts the timer.
    """
    code, fault = fb.get_fault(sensor)
    if code != 200 or fault is None:
        return f"Fault '{sensor}' not found."

    now_iso = datetime.now().isoformat()
    fb.update_fault(sensor, {"repair_start_time": now_iso, "status": "Repair Started"})
    return f"Repair for '{sensor}' started at {now_iso}."

def calculate_fault_score(fault):
    severity_multipliers = {
        "Low": 1,
        "Medium": 2,
        "High": 3,
        "Critical": 5
    }

    severity = fault.get("severity", "Low")
    multiplier = severity_multipliers.get(severity, 1)

    try:
        start_time = datetime.fromisoformat(fault["repair_start_time"])
        end_time = datetime.now()
        minutes_passed = max(1, (end_time - start_time).total_seconds() / 60)
    except Exception:
        minutes_passed = 1

    score = round(multiplier * (60 / minutes_passed))
    return max(1, score)

def complete_repair(sensor: str, username: str):
    """
    Completes the repair, updates score, and marks fault as repaired.
    """
    code, fault = fb.get_fault(sensor)
    if code != 200 or fault is None:
        return f"Fault '{sensor}' not found."

    score = calculate_fault_score(fault)

    fb.mark_fault_as_repaired(sensor, username)

    code, user_data = fb.get_user(username)
    if code != 200 or user_data is None:
        return f"User '{username}' not found."

    current_score = user_data.get("score", 0)
    new_score = current_score + score
    fb.update_user(username, {"score": new_score})

    return f"✅ Fault '{sensor}' repaired by {username}. +{score} XP (Total: {new_score})."